In [1]:
from monai.networks.nets import ViT as Vit_monai
from vit_pytorch.vit_3d import ViT as Vit
from monai.networks.nets import UNETR

In [2]:
print('asdasfdfsdfgsdg')
print('this is a test ddddddddddddddddddddddddddddddddd',flush=True)
print('this is a test',flush=True)

asdasfdfsdfgsdg
this is a test ddddddddddddddddddddddddddddddddd
this is a test


In [3]:
!nvidia-smi

Fri Jan  6 06:32:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:B1:00.0 Off |                  N/A |
| 20%   19C    P8     7W / 250W |      2MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:B2:00.0 Off |                  N/A |
| 24%   

In [4]:
import torch
import torch.nn as nn
from monai.networks.nets import UNETR

In [5]:
from monai.networks.blocks.unetr_block import UnetrBasicBlock, UnetrPrUpBlock, UnetrUpBlock

In [6]:
feature_size = 16
norm_name = "instance"
res_block = True
upblock = UnetrUpBlock(
            spatial_dims=3,
            in_channels=feature_size * 2,
            out_channels=feature_size,
            kernel_size=3,
            upsample_kernel_size=2,
            norm_name=norm_name,
            res_block=res_block,
        )

In [7]:
dec1 = torch.randn(1,32,16,80,80)
enc1 = torch.randn(1,16,32,160,160)
upblock(dec1, enc1).shape

torch.Size([1, 16, 32, 160, 160])

In [8]:
dec1 = torch.randn(1, 32, 64, 80, 80)

In [9]:
upblock.transp_conv(dec1).shape

torch.Size([1, 16, 128, 160, 160])

In [10]:
import sys
sys.path.append('/code/SuperHuman-main/scripts/scripts')
from model_superhuman import UNet_PNI

In [11]:
# 读取cfg
import yaml
from attrdict import AttrDict
from unetr import UNETR
import torch
cfg_file = 'seg_3d.yaml'
with open('/code/SuperHuman-main/scripts/scripts/config/' + cfg_file, 'r') as f:
        cfg = AttrDict(yaml.safe_load(f))


In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
superhuman =  UNet_PNI(in_planes=cfg.MODEL.input_nc,
                        out_planes=cfg.MODEL.output_nc,
                        filters=cfg.MODEL.filters,
                        upsample_mode=cfg.MODEL.upsample_mode,
                        decode_ratio=cfg.MODEL.decode_ratio,
                        merge_mode=cfg.MODEL.merge_mode,
                        pad_mode=cfg.MODEL.pad_mode,
                        bn_mode=cfg.MODEL.bn_mode,
                        relu_mode=cfg.MODEL.relu_mode,
                        init_mode=cfg.MODEL.init_mode).to(device)

In [13]:
superhuman

UNet_PNI(
  (embed_in): Sequential(
    (0): Conv3d(1, 28, kernel_size=(1, 5, 5), stride=(1, 1, 1), padding=(0, 2, 2))
    (1): ELU(alpha=1.0, inplace=True)
  )
  (conv0): resBlock_pni(
    (block1): Sequential(
      (0): Conv3d(28, 28, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
      (1): BatchNorm3d(28, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True)
      (2): ELU(alpha=1.0, inplace=True)
    )
    (block2): Sequential(
      (0): Conv3d(28, 28, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (1): BatchNorm3d(28, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True)
      (2): ELU(alpha=1.0, inplace=True)
      (3): Conv3d(28, 28, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    )
    (block3): BatchNorm3d(28, eps=1e-05, momentum=0.001, affine=True, track_running_stats=True)
    (block4): ELU(alpha=1.0, inplace=True)
  )
  (pool0): MaxPool3d(kernel_size=(1, 2, 2), str

In [14]:
unetr_path = '/data/cyd0806/superhuman/models/UNETR32*160*160_patch8*16*16_1230/2022-12-29--16-51-31_seg_3d/model-090000.pt'
unetr_states = torch.load(unetr_path)
unetr_states_dict = unetr_states['model_weights']
unetr_states_dict_new = {}
for key in unetr_states_dict.keys():
    unetr_states_dict_new[key[7:]] = unetr_states_dict[key]

vit_state = torch.load('/data/cyd0806/EM_raw_image/MAE/32*160*160_4*16*16_batch_1_mask0.75_1231/mae_vit_model_1220_32*160*160_200.pth')
vit_state_dict = vit_state['model_weights']
vit_state_dict_new = {}
for key in vit_state_dict.keys():
    vit_state_dict_new['vit.'+key] = vit_state_dict[key]



In [15]:
unetr = UNETR(
                        in_channels=cfg.MODEL.input_nc,
                        out_channels=cfg.MODEL.output_nc,
                        img_size=cfg.MODEL.unetr_size,
                        patch_size=(4,16,16),
                        kernel_size=(1,3,3),
                        feature_size=16,
                        hidden_size=768,
                        mlp_dim=3072,
                        num_heads=12,
                        pos_embed='perceptron',
                        norm_name='instance',
                        conv_block=True,
                        res_block=True,
                        dropout_rate=0.1).cuda()
unetr.load_state_dict(vit_state_dict_new,strict=False)

_IncompatibleKeys(missing_keys=['encoder1.layer.conv1.conv.weight', 'encoder1.layer.conv2.conv.weight', 'encoder1.layer.conv3.conv.weight', 'encoder2.transp_conv_init.conv.weight', 'encoder2.blocks.0.0.conv.weight', 'encoder2.blocks.0.1.conv1.conv.weight', 'encoder2.blocks.0.1.conv2.conv.weight', 'encoder2.blocks.1.0.conv.weight', 'encoder2.blocks.1.1.conv1.conv.weight', 'encoder2.blocks.1.1.conv2.conv.weight', 'encoder3.transp_conv_init.conv.weight', 'encoder3.blocks.0.0.conv.weight', 'encoder3.blocks.0.1.conv1.conv.weight', 'encoder3.blocks.0.1.conv2.conv.weight', 'encoder4.transp_conv_init.conv.weight', 'decoder5.transp_conv.conv.weight', 'decoder5.conv_block.conv1.conv.weight', 'decoder5.conv_block.conv2.conv.weight', 'decoder5.conv_block.conv3.conv.weight', 'decoder4.transp_conv.conv.weight', 'decoder4.conv_block.conv1.conv.weight', 'decoder4.conv_block.conv2.conv.weight', 'decoder4.conv_block.conv3.conv.weight', 'decoder3.transp_conv.conv.weight', 'decoder3.conv_block.conv1.conv.

In [16]:
x_in = torch.randn(1,1,32,160,160).cuda()
x, hidden_states_out,_ = unetr.vit(x_in)
enc1 = unetr.encoder1(x_in)
x2 = hidden_states_out[3]
enc2 = unetr.encoder2(unetr.proj_feat(x2, unetr.hidden_size, unetr.feat_size))
x3 = hidden_states_out[6]
enc3 = unetr.encoder3(unetr.proj_feat(x3, unetr.hidden_size, unetr.feat_size))
x4 = hidden_states_out[9]
enc4 = unetr.encoder4(unetr.proj_feat(x4, unetr.hidden_size, unetr.feat_size))
dec4 = unetr.proj_feat(x, unetr.hidden_size, unetr.feat_size)
dec3 = unetr.decoder5(dec4, enc4)
dec2 = unetr.decoder4(dec3, enc3)
dec1 = unetr.decoder3(dec2, enc2)
if unetr.patch_size[0] != unetr.patch_size[1]:
    dec1 = unetr.dtrans(dec1)
out = unetr.decoder2(dec1, enc1)
logits = unetr.out(out)

In [17]:
logits.shape

torch.Size([1, 3, 32, 160, 160])

In [18]:
import torch.nn as nn
unetr.decoder5.transp_conv = nn.ConvTranspose3d(768, 128, kernel_size=(1, 2, 2), stride=(1, 2, 2)).cuda()

In [19]:
dec4.shape

torch.Size([1, 768, 8, 10, 10])

In [20]:
unetr.decoder5.transp_conv(dec4).shape

torch.Size([1, 128, 8, 20, 20])

In [21]:
pretrained_params = []
other_params = []
for name,weight in unetr.named_parameters():
    if name not in vit_state_dict_new.keys():
        pretrained_params.append(weight)
    else:
        other_params.append(weight)

In [22]:
pretrained_params = []
other_params = []
for name,weight in unetr.named_parameters():
    if name not in vit_state_dict_new.keys():
        pretrained_params.append(weight)
    else:
        other_params.append(weight)

In [23]:
# 设置不同学习率
optimizer = torch.optim.Adam([
    {'params': pretrained_params, 'lr': 1e-4},
    {'params': other_params, 'lr': 1e-3}
])


In [24]:
for group in optimizer.param_groups:
    print(group['lr'])

0.0001
0.001


In [25]:
import argparse
from attrdict import AttrDict

# parser = argparse.ArgumentParser()
# parser.add_argument('--learning_rate', type=float, default=1e-3)
# parser.add_argument('--weight_decay', type=float, default=1e-4)
# parser.add_argument('--momentum', type=float, default=0.9)
# args = parser.parse_args()

args = AttrDict()
args.learning_rate = 1e-3
args.weight_decay = 1e-4
args.momentum = 0.9


all_params = unetr.parameters()
weight_params = []
quant_params = []
# 根据自己的筛选规则 将所有网络参数进行分组
for pname, p in unetr.named_parameters():
    if any([pname.endswith(k) for k in ['cw', 'dw', 'cx', 'dx', 'lamb']]):
        quant_params += [p]
    elif ('conv' or 'fc' in pname and 'weight' in pname):
        weight_params += [p]    
# 取回分组参数的id
params_id = list(map(id, weight_params)) + list(map(id, quant_params))
# 取回剩余分特殊处置参数的id
other_params = list(filter(lambda p: id(p) not in params_id, all_params))
# 构建不同学习参数的优化器
optimizer = torch.optim.SGD([
        {'params': other_params}, 
        {'params': quant_params, 'lr': 0.1*args.learning_rate},
        {'params': weight_params, 'weight_decay': args.weight_decay}],
        lr=args.learning_rate,
        momentum=args.momentum,
)


In [26]:
len(weight_params)

175

In [27]:
unetr.decoder5.transp_conv(torch.randn(1,768,4,16,16).cuda()).shape

torch.Size([1, 128, 4, 32, 32])

In [28]:
unetr.decoder5.transp_conv

ConvTranspose3d(768, 128, kernel_size=(1, 2, 2), stride=(1, 2, 2))

In [29]:
unetr.encoder4.transp_conv_init(torch.randn(1, 768, 8, 10, 10).cuda()).shape

torch.Size([1, 128, 16, 20, 20])

In [30]:
x_test = torch.randn(1,124,16,20,20).cuda()
for blk in unetr.encoder4.blocks:
    x_test = blk(x_test)
    print(x_test.shape)
#unetr.encoder2.blocks(torch.randn(1,32,16,20,20).cuda()).shape


In [31]:
unetr.proj_feat(x2, unetr.hidden_size, unetr.feat_size).shape

torch.Size([1, 768, 8, 10, 10])

In [32]:
unetr.decoder2

UnetrUpBlock(
  (transp_conv): Convolution(
    (conv): ConvTranspose3d(32, 16, kernel_size=(2, 2, 2), stride=(2, 2, 2), bias=False)
  )
  (conv_block): UnetResBlock(
    (conv1): Convolution(
      (conv): Conv3d(32, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
    )
    (conv2): Convolution(
      (conv): Conv3d(16, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
    )
    (lrelu): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm1): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (norm2): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (conv3): Convolution(
      (conv): Conv3d(32, 16, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
    )
    (norm3): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  )
)

In [33]:
unetr.decoder2.transp_conv(torch.randn(1,32,64,80,80).cuda()).shape

torch.Size([1, 16, 128, 160, 160])

In [34]:
int(unetr.patch_size[1]/unetr.patch_size[0])

4

In [35]:
nn.Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(int(unetr.patch_size[1]/unetr.patch_size[0]), 1, 1), padding=(1, 1, 1), bias=False)(torch.randn(1,32,64,80,80)).shape

torch.Size([1, 32, 16, 80, 80])

In [36]:
x_in = torch.randn(1,1,32,160,160).cuda() # patch_size=(4,16,16)
x, hidden_states_out,_ = unetr.vit(x_in) #(1,800,768) 
enc1 = unetr.encoder1(x_in) # torch.Size([1, 16, 32, 160, 160])
x2 = hidden_states_out[3] # torch.Size([1, 800, 768])
enc2 = unetr.encoder2(unetr.proj_feat(x2, unetr.hidden_size, unetr.feat_size)) # torch.Size([1, 32, 64, 80, 80])
x3 = hidden_states_out[6] # torch.Size([1, 800, 768])
enc3 = unetr.encoder3(unetr.proj_feat(x3, unetr.hidden_size, unetr.feat_size)) # torch.Size([1, 64, 32, 40, 40])
x4 = hidden_states_out[9] # torch.Size([1, 800, 768])
enc4 = unetr.encoder4(unetr.proj_feat(x4, unetr.hidden_size, unetr.feat_size)) # torch.Size([1, 128, 16, 20, 20])
dec4 = unetr.proj_feat(x, unetr.hidden_size, unetr.feat_size) # torch.Size([1, 768, 8, 10, 10])
dec3 = unetr.decoder5(dec4, enc4) # torch.Size([1, 128, 16, 20, 20])
dec2 = unetr.decoder4(dec3, enc3) # torch.Size([1, 64, 32, 40, 40])
dec1 = unetr.decoder3(dec2, enc2) # torch.Size([1, 32, 64, 80, 80])
out = unetr.decoder2(dec1, enc1)
logits = unetr.out(out)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 8 but got size 16 for tensor number 1 in the list.

In [ ]:
shape1 = unetr.proj_feat(x2, unetr.hidden_size, unetr.feat_size).shape
print(shape1)
test_data = torch.randn(shape1).cuda()
shape2 = unetr.encoder2.transp_conv_init(torch.randn(test_data.shape).cuda()).shape
print(shape2)
test_data2 = unetr.encoder2.transp_conv_init(torch.randn(test_data.shape).cuda())
for blk in unetr.encoder2.blocks:
    test_data2 = blk(test_data2)
    print(test_data2.shape)

torch.Size([1, 768, 8, 10, 10])
torch.Size([1, 32, 16, 20, 20])
torch.Size([1, 32, 32, 40, 40])
torch.Size([1, 32, 64, 80, 80])


In [ ]:
shape1 = unetr2.proj_feat(x2, unetr2.hidden_size, unetr2.feat_size).shape
print(shape1)
test_data = torch.randn(shape1).cuda()
shape2 = unetr2.encoder2.transp_conv_init(torch.randn(test_data.shape).cuda()).shape
print(shape2)
test_data2 = unetr2.encoder2.transp_conv_init(torch.randn(test_data.shape).cuda())
for blk in unetr2.encoder2.blocks:
    test_data2 = blk(test_data2)
    print(test_data2.shape)

torch.Size([1, 768, 2, 10, 10])
torch.Size([1, 32, 4, 20, 20])
torch.Size([1, 32, 8, 40, 40])
torch.Size([1, 32, 16, 80, 80])


In [ ]:
unetr2 = UNETR(
                        in_channels=cfg.MODEL.input_nc,
                        out_channels=cfg.MODEL.output_nc,
                        img_size=cfg.MODEL.unetr_size,
                        patch_size=(16,16,16),
                        feature_size=16,
                        hidden_size=768,
                        mlp_dim=3072,
                        num_heads=12,
                        pos_embed='perceptron',
                        norm_name='instance',
                        conv_block=True,
                        res_block=True,
                        dropout_rate=0.1).cuda()

In [ ]:
unetr2.proj_feat(x2, unetr2.hidden_size, unetr2.feat_size).shape

torch.Size([1, 768, 2, 10, 10])

In [ ]:
x_in = torch.randn(1,1,32,160,160).cuda()
x, hidden_states_out,_ = unetr2.vit(x_in) # torch.Size([1, 200, 768])
enc1 = unetr2.encoder1(x_in) # torch.Size([1, 16, 32, 160, 160])
x2 = hidden_states_out[3] # torch.Size([1, 200, 768])
enc2 = unetr2.encoder2(unetr2.proj_feat(x2, unetr2.hidden_size, unetr2.feat_size)) # torch.Size([1, 32, 16, 80, 80])
x3 = hidden_states_out[6] # torch.Size([1, 200, 768])
enc3 = unetr2.encoder3(unetr2.proj_feat(x3, unetr2.hidden_size, unetr2.feat_size)) # torch.Size([1, 64, 8, 40, 40])
x4 = hidden_states_out[9] # torch.Size([1, 200, 768])
enc4 = unetr2.encoder4(unetr2.proj_feat(x4, unetr2.hidden_size, unetr2.feat_size)) # torch.Size([1, 128, 4, 20, 20])
dec4 = unetr2.proj_feat(x, unetr2.hidden_size, unetr2.feat_size) # torch.Size([1, 768, 2, 10, 10])
dec3 = unetr2.decoder5(dec4, enc4) # torch.Size([1, 128, 4, 20, 20])
dec2 = unetr2.decoder4(dec3, enc3) # torch.Size([1, 64, 8, 40, 40])
dec1 = unetr2.decoder3(dec2, enc2) # torch.Size([1, 32, 16, 80, 80])
out = unetr2.decoder2(dec1, enc1) # torch.Size([1, 16, 32, 160, 160])
logits = unetr2.out(out) # torch.Size([1, 3, 32, 160, 160])

In [ ]:
unetr.encoder2.transp_conv_init(x2).shape

torch.Size([1, 32, 16, 20, 20])

In [ ]:
monai_vit = unetr.vit
mae_vit = Vit(
    image_size = 160,          # image size
    frames = 32,               # number of frames
    image_patch_size = 16,     # image patch size
    frame_patch_size = 4,      # frame patch size
    channels=1,
    num_classes = 1000,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)

In [ ]:
import torch
from vit_pytorch.vit_3d import ViT as Vit
from vit_pytorch import MAE
x = torch.randn(1,1,32,160,160)

In [ ]:
model = Vit(
                        image_size = 160,          # image size
                        frames = 32,               # number of frames
                        image_patch_size = 16,     # image patch size
                        frame_patch_size = 4,      # frame patch size
                        channels=1,
                        num_classes = 1000,
                        dim = 768,
                        depth = 12,
                        heads = 12,
                        mlp_dim = 3072,
                        dropout = 0.1,
                        emb_dropout = 0.1
                    )

mae = MAE(
    encoder = model,
    masking_ratio = 0.75,   # the paper recommended 75% masked patches
    decoder_dim = 512,      # paper showed good results with just 512
    decoder_depth = 6       # anywhere from 1 to 8
)

In [ ]:
model.pos_embedding.shape

torch.Size([1, 801, 768])

In [ ]:
!nvidia-smi

Wed Dec 28 19:39:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:60:00.0 Off |                  N/A |
| 25%   35C    P2    70W / 250W |   5187MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:61:00.0 Off |                  N/A |
| 24%   